## Bitcoin Prediction
#### Model to predict the price of Bitcoin (BTC)
#### Machine Learning Algorithm: **XGBoost**, **LSTM**, **SVM**

## Import Libraries

In [1]:
import pandas as pd
import seaborn as sns

import json
import calendar

import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import f1_score, classification_report, ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xgboost'